**Bank Renege:**
This models a bank counter and customers arriving t random times and with each customer having a certain patience. It
waits to get to the counter until she’s at the end of her tether. If she gets to the counter, she uses it for a while before
releasing it.

In [14]:
%pip install simpy

In [19]:
import random
import simpy
RANDOM_SEED = 42
NEW_CUSTOMERS = 10 # Total number of customers
INTERVAL_CUSTOMERS = 15.0 # Generate new customers roughly every x seconds
MIN_PATIENCE = 1 # Min. customer patience
MAX_PATIENCE = 3 # Max. customer patience

In [20]:
def source(env, number, interval, counter):
  """Source generates customers randomly"""
  for i in range(number):
    c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
    env.process(c)
    t = random.expovariate(1.0 / interval)
    yield env.timeout(t)

In [21]:
def customer(env, name, counter, time_in_bank):
  """Customer arrives, is served and leaves."""
  arrive = env.now
  print('%7.4f %s: Arrived' % (arrive, name))
  with counter.request() as req:
    patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
    # Wait for the counter or abort at the end of our tether
    results = yield req | env.timeout(patience)
    wait = env.now - arrive

    if req in results:
      # We got to the counter
      print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
      tib = random.expovariate(1.0 / time_in_bank)
      yield env.timeout(tib)
      print('%7.4f %s: Finished' % (env.now, name))
    else:
      # We reneged
      print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))

In [22]:
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Arrived
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
15.3009 Customer01: Arrived
15.3009 Customer01: Waited  0.000
19.0897 Customer02: Arrived
20.2636 Customer02: RENEGED after  1.174
28.8510 Customer01: Finished
52.4990 Customer03: Arrived
52.4990 Customer03: Waited  0.000
55.4596 Customer03: Finished
60.7197 Customer04: Arrived
60.7197 Customer04: Waited  0.000
63.3800 Customer04: Finished
71.2784 Customer05: Arrived
71.2784 Customer05: Waited  0.000
74.2668 Customer05: Finished
87.0208 Customer06: Arrived
87.0208 Customer06: Waited  0.000
87.0991 Customer06: Finished
100.3680 Customer07: Arrived
100.3680 Customer07: Waited  0.000
105.3587 Customer07: Finished
124.9524 Customer08: Arrived
124.9524 Customer08: Waited  0.000
127.4872 Customer09: Arrived
128.6807 Customer09: RENEGED after  1.193
129.8769 Customer08: Finished
